<a href="https://colab.research.google.com/github/xuxiufeng/Spam_ham_Classification/blob/main/Spam_ham_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip enron1_test.zip
!unzip enron1_train.zip
!unzip enron4_test.zip
!unzip enron4_train.zip
!unzip hw1_test.zip
!unzip hw1_train.zip

Archive:  enron1_test.zip
   creating: enron1/test/
   creating: enron1/test/ham/
  inflating: enron1/test/ham/0015.1999-12-15.farmer.ham.txt  
  inflating: enron1/test/ham/0024.1999-12-16.farmer.ham.txt  
  inflating: enron1/test/ham/0025.1999-12-16.farmer.ham.txt  
  inflating: enron1/test/ham/0043.1999-12-21.farmer.ham.txt  
  inflating: enron1/test/ham/0086.1999-12-29.farmer.ham.txt  
  inflating: enron1/test/ham/0091.1999-12-29.farmer.ham.txt  
  inflating: enron1/test/ham/0111.1999-12-31.farmer.ham.txt  
  inflating: enron1/test/ham/0129.2000-01-05.farmer.ham.txt  
  inflating: enron1/test/ham/0156.2000-01-10.farmer.ham.txt  
  inflating: enron1/test/ham/0166.2000-01-11.farmer.ham.txt  
  inflating: enron1/test/ham/0183.2000-01-12.farmer.ham.txt  
  inflating: enron1/test/ham/0213.2000-01-18.farmer.ham.txt  
  inflating: enron1/test/ham/0215.2000-01-18.farmer.ham.txt  
  inflating: enron1/test/ham/0223.2000-01-19.farmer.ham.txt  
  inflating: enron1/test/ham/0242.2000-01-21.farme

In [8]:
import sys
import collections
import os 
import re
import codecs
import numpy

In [11]:
stop_words = ["a","about","above","after","again","against","all","am","an","and",
"any","are","aren't","as","at","be","because","been","before","being","below",
"between","both","but","by","can't","cannot","could","couldn't","did","didn't",
"do","does","doesn't","doing","don't","down","during","each","few","for","from",
"further","had","hadn't","has","hasn't","have","haven't","having","he","he'd",
"he'll","he's","her","here","here's","hers","herself","him","himself","his","how",
"how's","i","i'd","i'll","i'm","i've","if","in","into","is","isn't","it","it's","its",
"itself","let's","me","more","most","mustn't","my","myself","no","nor","not","of",
"off","on","once","only","or","other","ought","our","ours","ourselves","out","over",
"own","same","shan't","she","she'd","she'll","she's","should","shouldn't","so","some",
"such","than","that","that's","the","their","theirs","them","themselves","then","there",
"there's","these","they","they'd","they'll","they're","they've","this","those","through",
"to","too","under","until","up","very","was","wasn't","we","we'd","we'll","we're","we've",
"were","weren't","what","what's","when","when's","where","where's","which","while","who",
"who's","whom","why","why's","with","won't","would","wouldn't","you","you'd","you'll",
"you're","you've","your","yours","yourself","yourselves"]

def ReadFile(file,filepath):
    fileHandler = codecs.open(filepath+"/" + file,'rU','latin-1')
    Findwords = re.findall('[A-Za-z0-9\']+', fileHandler.read())
    allwords = list()
    for word in Findwords:
        word = word.lower()
        allwords+=[word]
    fileHandler.close()    
    return allwords
    
def GetWordListsAndNumberOffiles(filepath):
    wordList = list()
    NumberOfFiles = 0
    for files in os.listdir(filepath):
        if files.endswith(".txt"):
            wordList += ReadFile(files,filepath)
            NumberOfFiles+=1
    return wordList, NumberOfFiles

def initiliazeMatrix(row, column):
    featureMatrix = [0] * row
    for i in range(row):
        featureMatrix[i] = [0] * column
    return featureMatrix

In [5]:
train_path = 'enron1/train'

In [6]:
#location of the folder for ham & spam for train and test

HamFolderPath = train_path + '/ham'
SpamFolderPath = train_path + '/spam'
    

In [12]:
#Find out all the words in ham folder and spam folder and find there counts  
NumberOfHam, NumberOfSpam = 0, 0
WordListInham = []
WordListInspam = []
WordListInham,NumberOfHam = GetWordListsAndNumberOffiles(HamFolderPath)
WordListInspam,NumberOfSpam = GetWordListsAndNumberOffiles(SpamFolderPath)